In [ ]:
from azure.ai.ml import MLClient, command, Input, Output
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

input_data_asset = ml_client.data.get("oor_velotech", version="1")
output_data_asset = ml_client.data.get("oor_velotech_frames", version="1")

# to successfully create a job, customize the parameters below based on your workspace resources
job = command(
    code="../",
    command='poetry run python objectherkenning_openbare_ruimte/frame_extraction/frame_extraction.py --input_folder "${{inputs.input_folder}}" --output_folder "${{outputs.output_folder}}"',
    inputs={
        "input_folder": Input(path=input_data_asset.id,
            type=AssetTypes.URI_FOLDER,
            mode=InputOutputModes.RO_MOUNT
        )
    },
    outputs={
        "output_folder": Output(path=output_data_asset.path,
            type=AssetTypes.URI_FOLDER,
            mode=InputOutputModes.RW_MOUNT
        )
    },
    compute="cpu-daan",
    environment="oor-test-environment:1"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)